In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from proofreader.utils.vis import plot_3d
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from proofreader.data.cremi import prepare_cremi_vols

train_vols, test_vols = prepare_cremi_vols('../../dataset/cremi')

In [19]:
from proofreader.data.splitter import SplitterDataset
from proofreader.data.augment import Augmentor
num_slices = [1, 4]
radius = 96
context_slices = 6
num_points = 1024

augmentor = Augmentor(center=True, shuffle=True, rotate=True, scale=True)
dataset = SplitterDataset(train_vols, num_slices, radius, context_slices, num_points=num_points, torch=False, open_vol=True, verbose=True, Augmentor=augmentor)
print(len(dataset))

3534


In [27]:
from proofreader.utils.vis import plot_3d
%matplotlib widget
num = 10
lim = (-.1,.1)
lim = (lim,lim,lim)
print(lim)
for i in range(0, num):
    rand_i = random.randint(0, len(dataset))
    pc, label = dataset[rand_i]
    pc = np.swapaxes(pc, 0, 1)
    plot_3d(pc, lims=lim)



((-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1))
1, vol: 0, c: 13236
num_slices: 1, drop: [50, 51]
pc shape: (3, 1024), vol shape: (12, 119, 144)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1, vol: 0, c: 6106
num_slices: 2, drop: [27, 29]
pc shape: (3, 1024), vol shape: (12, 69, 94)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0, vol: 1, c: 514
num_slices: 2, drop: [40, 42]
pc shape: (3, 1024), vol shape: (14, 172, 156)


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0, vol: 1, c: 522
num_slices: 2, drop: [55, 57]
pc shape: (3, 1024), vol shape: (12, 114, 419)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1, vol: 2, c: 1959
num_slices: 1, drop: [104, 105]
pc shape: (3, 1024), vol shape: (9, 138, 180)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …